# Information Extraction using Python

Nos inputs en tant que données textuelles contiennent certaines informations, mais elles ne sont pas toutes importantes pour nous. Nous devons donc extraire dans ces données les informations pertinentes nous aidant à atteindre notre objectif. Dans notre cas nous devons déterminer quelles sont les gares de départ et de d'arrivée du trajet dont notre utilisateur veut connaitre les informations pratiques.

ex: "Je recherche un train depuis Paris pour Lyon"

Dans ce cas là les informations qui nous intéressent sont "Paris" et "Lyon". 

Ce qui nous intéresse également c'est la relation qui existe entre les deux entités. Dans notre cas il s'agit de la relation "départ/arrivée". 

In [ ]:
# Installer les dépendances avant d'executer les cellules

#!pip install spacy-lefff

In [ ]:
#!python -m spacy download fr_core_news_sm

# Avant d'aller plus loin => Exécution > Redémarrer l'environnement d'execution

## Méthode 

À partir d'un segment de texte nous souhaitons trouver les localisation qui y sont mentionnées. 

### Bref résumé du fonctionnement de l'algorithme: 

1) Extraction des potentielles localisation dans le texte. 

2) Chercher pour chaque potentielle localisation une référence dans une base de connaissance. 



# Location extraction

Pour cette première phase nous allons utiliser deux process: Part-Of-Speech (POS) tagger & named
entity recognizer (NER). 

L'intégralité du segment de texte va être taggé avec ces deux models. Nous allons donc obtenir une ensemble de terme qui vont représenter les potentielles location contenue dans notre segment. 

## L’approche NER (Named Entity Recognition)

In [ ]:
import spacy
from spacy import displacy

In [ ]:
nlp = spacy.load("fr_core_news_sm")
doc = nlp(u"Je recherche un train depuis reims jusqu'à Lyon")

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

## Part of Speech Tagger

Nous allons maintenant analyser la composition de d'une phrase.

Nous allons déterminer le tag POS de chaque mot dans notre phrase: "Je recherche un train depuis Paris pour Lyon". 

In [ ]:
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger
from spacy.language import Language


@Language.factory('french_lemmatizer')
def create_french_lemmatizer(nlp, name):
    return LefffLemmatizer(after_melt=True, default=True)

@Language.factory('melt_tagger')  
def create_melt_tagger(nlp, name):
    return POSTagger()

nlp = spacy.load("fr_core_news_sm")
nlp.add_pipe('melt_tagger', after='parser')
nlp.add_pipe('french_lemmatizer', after='melt_tagger')

100%|██████████| 88.9M/88.9M [00:03<00:00, 25.3MB/s]


In [ ]:
# Result of POS tagging

doc = nlp(u"Je recherche un train depuis Paris jusqu'à Lyon")
doc3 = nlp(u"Je recherche un train depuis Gare de Lyon jusqu'à Lyon")
for d in doc:
    print(d.text, ' -> ', d.pos_)

for d in doc3:
    print(d.text, ' -> ', d.pos_)

Je  ->  PRON
recherche  ->  VERB
un  ->  DET
train  ->  NOUN
depuis  ->  ADP
Paris  ->  PROPN
jusqu'  ->  ADP
à  ->  ADP
Lyon  ->  PROPN
Je  ->  PRON
recherche  ->  VERB
un  ->  DET
train  ->  NOUN
depuis  ->  ADP
Gare  ->  NOUN
de  ->  ADP
Lyon  ->  PROPN
jusqu'  ->  ADP
à  ->  ADP
Lyon  ->  PROPN


In [ ]:
# Result of NER tagging 
for ent in doc3.ents:
    print(ent.text, ent.label_)

Gare de Lyon LOC
Lyon LOC


Nous pouvons donc maintenant tenter d'extraire seulement les noms propres que contient notre phrase pour obtenir les informations qui nous intéressent (les villes de départ et d'arrivée)

In [ ]:
for d in doc:
  if d.pos_ =='PROPN':
    print(d.text)

Paris
Lyon


### Quelle est la relation entre les tokens de ma phrase ? 

In [ ]:
from spacy import displacy 

options = { "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro"}

displacy.render(doc, style='dep',jupyter=True, options=options)

## Le problème avec les noms de gare 

Se baser sur le POS tag 'PROPN' ne suffit pas pour reconnaitre le point de départ et d'arrivée. En effet les noms de gare ne sont pas reconnu comme des noms propres par le POS tagger. 

In [ ]:
doc2 = nlp(u"Quel est le prochain train pour Reims depuis gare de l'Est")

for d in doc2:
  if d.pos_ =='PROPN':
    print(d.text)

displacy.render(doc2, style='dep',jupyter=True, options=options)

Reims
